In [87]:
graph = {
    'aranos': {'barlow': 14, 'daxx': 7, 'yeedil': 9},
    'boldan': {'barlow': 9, 'oozla': 6},
    'barlow': {'aranos': 14, 'boldan': 9, 'yeedil': 2},
    'daxx': {'aranos': 7, 'yeedil': 10, 'oozla': 16},
    'yeedil': {'aranos': 9, 'barlow': 2, 'daxx': 10, 'oozla': 11},
    'oozla': {'boldan': 6, 'daxx': 15, 'yeedil': 11},
}


In [88]:
class Edge:
    def __init__(self, weight, vertex):
        self.vertex = vertex
        self.weight = weight

    @property
    def vertex(self):
        return self.__vertex

    @vertex.setter
    def vertex(self, vertex):
        self.__vertex = vertex

    @property
    def weight(self):
        return self.__weight

    @weight.setter
    def weight(self, weight):
        self.__weight = weight

    def __str__(self):
        return "(" + str(self.weight) + ") -> " + str(self.vertex.name)

    def __eq__(self, other):
        if other:
            if isinstance(other, Edge):
                return self.weight == other.weight
        else:
            return False

    def __ne__(self, other):
        if other:
            if isinstance(other, Edge):
                return self.weight != other.weight
        else:
            return False

    def __lt__(self, other):
        if other:
            if isinstance(other, Edge):
                return self.weight < other.weight
        else:
            return False

    def __le__(self, other):
        if other:
            if isinstance(other, Edge):
                return self.weight <= other.weight
        else:
            return False

    def __gt__(self, other):
        if other:
            if isinstance(other, Edge):
                return self.weight > other.weight
        else:
            return False

    def __ge__(self, other):
        if other:
            if isinstance(other, Edge):
                return self.weight >= other.weight
        else:
            return False

In [89]:
class Vertex:
    def __init__(self, name, adjecencyList=[]):
        self.name = name
        self.adjecent = adjecencyList
        self.distance = None
        self.previous = None
        self.known = False

    @property
    def name(self):
        return self.__name

    @name.setter
    def name(self, name):
        self.__name = name

    @property
    def adjecent(self):
        return self.__adjecencylist

    @adjecent.setter
    def adjecent(self, adjecencylist):
        self.__adjecencylist = adjecencylist

    @property
    def distance(self):
        return self.__distance

    @distance.setter
    def distance(self, distance):
        self.__distance = distance

    @property
    def previous(self):
        return self.__previous

    @previous.setter
    def previous(self, previous_node):
        self.__previous = previous_node

    @property
    def known(self):
        return self.__known

    @known.setter
    def known(self, known):
        self.__known = known

    def add_adjecent_edge(self, edge=None):
        if edge:
            if isinstance(edge, Edge):
                if self.adjecent == []:
                    self.adjecent = [edge]
                else:
                    self.adjecent.append(edge)
        else:
            raise TypeError(
                "Attempt to add something in adjecencylist which is not an edge"
            )

    def __str__(self):
        retval = str(self.name) + ": ["
        for e in self.adjecent:
            retval += e.vertex.name + "(" + str(e.weight) + ")"
        retval += (
            "](known = " + str(self.known) + ")(distance = " + str(self.distance) + ")"
        )
        if self.distance:
            retval += "(previous='" + str(self.previous.name) + "')"
        return retval

In [90]:
class Graph:
    def __init__(self):
        self.__vertecies = {}

    @property
    def vertecies(self):
        return self.__vertecies

    def addEdge(self, from_v=None, to_v=None, weight=1):
        if from_v and to_v:
            if from_v not in self.__vertecies:
                self.__vertecies[from_v] = Vertex(name=from_v)
            if to_v not in self.__vertecies:
                self.__vertecies[to_v] = Vertex(name=to_v)
            edge = Edge(weight, self.__vertecies[to_v])

            self.__vertecies[from_v].add_adjecent_edge(edge)
        else:
            raise Exception("Illegal edge definition")

    def __str__(self):
        retval = ""
        for v in self.vertecies:
            vertex = g.vertecies[v]
            retval += v + "(" + str(vertex.distance) + ")"
            for e in vertex.adjecent:
                retval += (
                    " --> ["
                    + e.vertex.name
                    + ": "
                    + str(e.weight)
                    + "]"
                    
                )
            retval += "\n"
        return retval
    
    def readGraph(self, graph):

        for f in graph:
            for t, w in graph[f].items():
                self.addEdge(f, t, w)


    def getPath(self, fromVertex, toVertex):
        # To verify if a path exists, the path has to be updated ...
        # Need to verify that the destination vertex exists within the graph
        if toVertex not in self.vertecies:
            raise KeyError("Destination node not present in graph")
        from queue import LifoQueue

        stack = LifoQueue()

        def push(data):
            stack.put(data)

        def pop():
            return stack.get()

        vertex = self.vertecies[toVertex]
        push(vertex)
        while True:
            if vertex.previous is not None:
                push(vertex.previous)
                vertex = vertex.previous
            else:
                break
        retval = []
        while not stack.empty():
            retval.append(pop())
        return retval

    def getPathAsString(self, fromVertex, toVertex):
        vertecies = self.getPath(fromVertex, toVertex)
        retval = ""
        for i in range(0, len(vertecies)):
            if vertecies[i] == vertecies[-1]:
                retval += vertecies[i].name + "\n" + "Total weight of path: " + str(vertecies[i].distance) + " "
            else:
                retval += vertecies[i].name + ": " +  str((vertecies[i+1].distance - vertecies[i].distance)) + " "
            if i < len(vertecies) - 1:
                retval += "--> "
        return retval

    def resetGraph(self):
        for v in self.vertecies:
            vertex = g.vertecies[v]
            vertex.previous = None
            vertex.distance = None
            vertex.known = False

    def Dijkstra(self, startVertexName):
        # Check to see that startvertex is in Graph
        if startVertexName not in self.vertecies:
            raise KeyError("Start node not present in graph")
        # Reset visited and previous pointer before running algorithm
        self.resetGraph()

        vertex = self.vertecies[startVertexName]
        vertex.distance = distance = weight = 0
        previous_node = None

        # Create priority queue, priority = current weight on edge ...
        # No duplicate edges is queue allowed
        edge = Edge(0, vertex)

        from queue import PriorityQueue

        priqueue = PriorityQueue()

        def enqueue(data):
            priqueue.put(data)

        def dequeue():
            return priqueue.get()

        enqueue(edge)
        while not priqueue.empty():
            # Get the element with lowest priority (i.e. weight on edge)
            edge = dequeue()
            eyeball = edge.vertex
            # If not visited previously, we need to define the distance
            if not eyeball.known:
                eyeball.distance = distance
                eyeball.previous = previous_node
            eyeball.known = True

            # If the vertex pointed to by the edge has an adjecency list, we need to iterate on it
            for adjecentedge in eyeball.adjecent:
                if not adjecentedge.vertex.known:
                    adjecentedge.vertex.distance = (
                        eyeball.distance + adjecentedge.weight
                    )
                    adjecentedge.vertex.previous = eyeball
                    adjecentedge.vertex.known = True
                    enqueue(adjecentedge)
                else:
                    if (
                        adjecentedge.vertex.distance
                        > eyeball.distance + adjecentedge.weight
                    ):
                        adjecentedge.vertex.distance = (
                            eyeball.distance + adjecentedge.weight
                        )
                        adjecentedge.vertex.previous = eyeball
                        enqueue(adjecentedge)
g = Graph()
g.readGraph(graph)
print(g)

aranos(None) --> [barlow: 14] --> [daxx: 7] --> [yeedil: 9]
barlow(None) --> [aranos: 14] --> [boldan: 9] --> [yeedil: 2]
daxx(None) --> [aranos: 7] --> [yeedil: 10] --> [oozla: 16]
yeedil(None) --> [aranos: 9] --> [barlow: 2] --> [daxx: 10] --> [oozla: 11]
boldan(None) --> [barlow: 9] --> [oozla: 6]
oozla(None) --> [boldan: 6] --> [daxx: 15] --> [yeedil: 11]



In [91]:
g.Dijkstra('aranos')
x = g.getPathAsString('aranos', 'boldan')
print("Shortest path from Aranos til Boldan:")
print(x)

Shortest path from Aranos til Boldan:
aranos: 9 --> yeedil: 2 --> barlow: 9 --> boldan
Total weight of path: 20 


In [92]:
g.Dijkstra('barlow')
x = g.getPathAsString('barlow', 'aranos')
print("Shortest path from Barlow til Aranos:")
print(x)

Shortest path from Barlow til Aranos:
barlow: 2 --> yeedil: 9 --> aranos
Total weight of path: 11 


In [93]:
g.Dijkstra('daxx')
x = g.getPathAsString('daxx', 'boldan')
print("Shortest path from Daxx til Boldan:")
print(x)

Shortest path from Daxx til Boldan:
daxx: 10 --> yeedil: 2 --> barlow: 9 --> boldan
Total weight of path: 21 
